### Extract some data about annotations from the 48 annotated Procure documents
  * Popular properties and items IDs in the annotations
  * How often the header cells were annotated with a **property** or **item** IDs
  * How often data cells were annotated with **property** or **item** IDs
  
Conclusions:
  * 93% of the 1st header column cell annotations were to **items**
  * 27% of the remaining header column cell annotations were to **items**
  * 99.9% of the data cell annotations were **items**
  * We should focus on properties for linking header cells and use the annotations to identify target/good properties for this domain that

In [91]:
import pandas as pd
from collections import defaultdict
import wd_search as wd
import os.path
from operator import itemgetter

In [92]:
dir = "/Users/finin/Projects/IARPA_GE/July_deliverable/Annotated_Table_Objects/Annotated_Tables"

In [93]:
# dictionaries for counrs of annotation ids in header row or data rows
header1_ids = defaultdict(int)
header_rest_ids = defaultdict(int)
header_ids = defaultdict(int)
data_ids = defaultdict(int)

total_columns = 0
total_data_cells = 0

In [94]:
# process each annotated document file
doc_count = 0
for file in os.scandir(dir):
    doc_count += 1
    # print(file.name)
    df = pd.read_excel(file.path, header=None)
    nrows, ncols = df.shape
    total_columns += int(ncols/2)
    total_data_cells += int(ncols/2) * (nrows - 1) 
    for row in range(nrows):
        # do odd columns starting with 1
        for col in range(1,ncols,2):
            id = df.iat[row, col]
            id = id.replace('procure_', '') if type(id)==str else id
            if row == 0:
                header_ids[id] += 1
                if col == 1: # also track 1st column header
                    header1_ids[id] +=1
                else:
                    header_rest_ids[id] +=1
            else:
                data_ids[id] += 1

In [95]:
# Is an annotation id to a single Qid or Pid?
def good_id(id):
    return ( type(id) == str) and id and (id[0] in "PQ") and (' ' not in id) and (';' not in id) and ("_" not in id)

def get_label(id):
    return wd.get_label(id) if good_id(id) else '-----'

### Compute some simple statistics

In [96]:
print(f"Processed {doc_count} sheets with {total_columns} columns and {total_data_cells} data cells ")
print(f"Found {len(header_ids)} unique annotations in header rows; {len(data_ids)} in data rows")

# annotation meta-types (Q or P) in 1st header column
header1_p = header1_q = good = 0
for (id, n) in header1_ids.items():
    if good_id(id):
        good += n
        if id[0]=='Q':
            header1_q += n
        else:
            header1_p += n
print(f"For 1st header cell, {100*(header1_p/good):.1f}% were properties; {100*(header1_q/good):.1f}% items")

# annotation meta-types (Q or P) in remaining header columns
header_rest_p = header_rest_q = good = 0
for (id, n) in header_rest_ids.items():
    if good_id(id):
        good += n
        if id[0]=='Q':
            header_rest_q += n
        else:
            header_rest_p += n
print(f"In other header cells {100*(header_rest_p/good):.1f}% were properties; {100*(header_rest_q/good):.1f}% items")


# annotation meta-types (Q or P) in data cells
data_p = data_q = data_good = 0
for (id, n) in data_ids.items():
    if good_id(id):
        data_good += n
        if id[0]=='Q':
            data_q += n
        else:
            data_p += n
print(f"In data cells {100*(data_p/data_good):.1f}% were properties; {100*(data_q/data_good):.1f}% items")


Processed 48 sheets with 302 columns and 3448 data cells 
Found 87 unique annotations in header rows; 346 in data rows
For 1st header cell, 8.1% were properties; 91.9% items
In other header cells 73.1% were properties; 26.9% items
In data cells 0.1% were properties; 99.9% items


### links found in header rows

In [97]:
for (id, n) in sorted(list(header_ids.items()), key=itemgetter(1), reverse=True):
    print(n, '\t', id, '\t', get_label(id))

70 	 NA 	 -----
17 	 P1603 	 number of cases
16 	 P1120 	 number of deaths
13 	 ref 	 -----
13 	 Q6256 	 country
12 	 P3457 	 case fatality rate
11 	 P7153 	 significant place
9 	 Q105967696 	 vaccine type
8 	 P585 	 point in time
7 	 Q58702 	 mortality rate
6 	 P1082 	 population
5 	 P1193 	 prevalence
5 	 id 	 -----
5 	 Q855769 	 strain
4 	 P6099 	 clinical trial phase
4 	 P_NA 	 -----
3 	 Q2221906 	 geographic location
3 	 P3629 	 age at event
3 	 Q18420531 	 viral strain
3 	 P3098 	 ClinicalTrials.gov ID
3 	 P2844 	 incidence
3 	 P2047 	 duration
3 	 P178 	 developer
3 	 Q87719492 	 COVID-19 vaccine
2 	 P21 	 sex or gender
2 	 P1114 	 quantity
2 	 Q89469904 	 coronavirus
2 	 P923 	 medical examinations
2 	 Q82794 	 geographic region
2 	 Q2520821 	 length of stay
2 	 P1545 	 series ordinal
2 	 Q13410403 	 county of Colorado
2 	 Q104778232 	 COVID-19 mortality
2 	 P780 	 symptoms and signs
1 	 nan 	 -----
1 	 P1724 	 
1 	 Q11494956 	 source of infection
1 	 Q5910902 	 Hot zone
1 	 P1

### Links found in first column of header row

In [98]:
for (id, n) in sorted(list(header1_ids.items()), key=itemgetter(1), reverse=True):
    print(n, '\t', id, '\t', get_label(id))

10 	 Q6256 	 country
5 	 id 	 -----
4 	 ref 	 -----
3 	 Q2221906 	 geographic location
2 	 Q18420531 	 viral strain
2 	 P585 	 point in time
2 	 Q82794 	 geographic region
2 	 Q87719492 	 COVID-19 vaccine
2 	 Q855769 	 strain
2 	 Q13410403 	 county of Colorado
1 	 NA 	 -----
1 	 Q161524 	 mask
1 	 P2598 	 serial number
1 	 Q42014143 	 biomedical measurand type
1 	 Q29512471 	 cancer types
1 	 Q515 	 city
1 	 Q8817226 	 Category:Statistical tests
1 	 Q28051899 	 vaccine candidate
1 	 Q11801904 	 PMCID
1 	 Q89469904 	 coronavirus
1 	 P1542; Q12136 	 -----
1 	 Q5119 	 capital
1 	 Q12184 	 pandemic
1 	 Q105967696 	 vaccine type


### links found in data rows more than twice

In [86]:
for (id, n) in sorted(list(data_ids.items()), key=itemgetter(1), reverse=True):
    if n < 3:
        break
    print(n, '\t', id, '\t', get_label(id))

2449 	 lit 	 -----
111 	 ref 	 -----
45 	 Q82069695 	 SARS-CoV-2
42 	 nan 	 -----
36 	 Q148 	 People's Republic of China
32 	 NA 	 -----
31 	 Q30 	 United States of America
18 	 Q145 	 United Kingdom
18 	 Q85438966 	 SARS-CoV-1
15 	 Q38 	 Italy
14 	 Q4902157 	 Middle East respiratory syndrome coronavirus
13 	 Q16654806 	 Middle East respiratory syndrome
13 	 Q106336308 	 viral vector vaccine
12 	 Q11746 	 Wuhan
12 	 Q183 	 Germany
12 	 Q85795487 	 RNA vaccine
11 	 Q142 	 France
11 	 Q159 	 Russia
11 	 Q1436668;
Q608510;
Q7200267 	 -----
11 	 Q432083 	 intramuscular injection
11 	 Q104783588 	 vector-based vaccine
10 	 Q851 	 Saudi Arabia
10 	 Q96381373 	 Human-to-human transmission
10 	 Q161393 	 influenza A virus subtype H1N1
9 	 Q876852 	 summer camp
7 	 Q3560939 	 inactivated vaccine
7 	 Q29 	 Spain
7 	 Q17 	 Japan
7 	 Q99 	 California
6 	 Q668 	 India
6 	 id 	 -----
6 	 Q182672 	 zoonosis
6 	 Q288514 	 Fair
6 	 Q408 	 Australia
6 	 Q794 	 Iran
5 	 Q884 	 South Korea
5 	 Q155 	 Braz

In [32]:
"fin"

'fin'